# US household debt indicators
> This notebook downloads, processes and charts data related to household debt from the US Federal Reserve. 

---

#### Import Python tools and Jupyter config

In [1]:
%pip install cpi

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import cpi
import requests
import pandas as pd
# import jupyter_black
import altair as alt
import geopandas as gpd
import altair_cnn as altcnn
from IPython.display import Image
from datawrapper import Datawrapper

/Users/almatthews/.pyenv/versions/3.9.16/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/q0/1clc4jf97cxf003x4xgh89680000gr/T/ipykernel_55315/4231823431.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapel

In [2]:
# jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None
alt.themes.register("cnn", altcnn.theme)
alt.themes.enable("cnn")

ThemeRegistry.enable('cnn')

In [3]:
dw_token = os.environ.get("dw_api")
dw = Datawrapper(access_token=dw_token)
today = pd.Timestamp("today").strftime("%Y-%m-%d")

---

## Fetch

#### Debt report from the [New York Fed Consumer Credit Panel/Equifax](https://www.newyorkfed.org/microeconomics/hhdc)

In [4]:
url = "https://www.newyorkfed.org/medialibrary/interactives/householdcredit/data/xls/HHD_C_Report_2024Q1"

In [5]:
xlsx_file = pd.read_excel(url, sheet_name="TABLE OF CONTENTS", skiprows=5, skipfooter=8)
sheet_names = pd.ExcelFile(url).sheet_names

---

#### Quarterly household debt service ratio
> This [measure](https://fred.stlouisfed.org/series/TDSP) is the ratio of total required household debt payments to total disposable income.

In [6]:
debt_ratio_url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id=TDSP"

In [7]:
debt_ratio_df = pd.read_csv(debt_ratio_url, names=["date", "value"], header=0).round(2)

In [8]:
debt_ratio_df

,date,value
0,1980-01-01,10.61
1,1980-04-01,10.63
2,1980-07-01,10.40
3,1980-10-01,10.25
4,1981-01-01,10.29
...,...,...
171,2022-10-01,9.89
172,2023-01-01,9.71
173,2023-04-01,9.70
174,2023-07-01,9.77


#### Last decade

In [9]:
recent = debt_ratio_df.query('date>"1999-12-31"').copy()

#### Export to Datawrapper [chart](https://app.datawrapper.de/chart/tXaoi/publish)

In [10]:
debt_ratio_id = "tXaoi"
dw.add_data(
    chart_id=f"{debt_ratio_id}",
    data=recent,
)

---

#### Percentage of debt 90+ days delinquent
> Aggregate delinquency rates increased in the fourth quarter of 2023. As of December, 3.1% of outstanding debt was in some stage of delinquency, up by 0.1 percentage point from the third quarter. Still, overall delinquency rates remain 1.6 percentage points lower than the fourth quarter of 2019.

In [11]:
del_df = (
    pd.read_excel(url, sheet_name="Page 12 Data", skiprows=3)
    .drop(["Unnamed: 8", "Unnamed: 9"], axis=1)
    .round(2)
).rename(
    columns={
        "Unnamed: 0": "quarter",
        "MORTGAGE": "mortgage",
        "HELOC": "heloc",
        "AUTO": "auto",
        "CC": "credit_card",
        "STUDENT LOAN": "student_loan",
        "OTHER": "other",
        "ALL": "all",
    }
)

#### Clean dates

In [12]:
del_df["year"] = "20" + del_df["quarter"].str.split(":", expand=True)[0]
del_df["quarter"] = del_df["quarter"].str.split(":", expand=True)[1]

In [13]:
quarter_to_date = {
    "Q1": "-01-01",  # January 1st
    "Q2": "-04-01",  # April 1st
    "Q3": "-07-01",  # July 1st
    "Q4": "-10-01",  # October 1st
}

In [14]:
del_df["date"] = pd.to_datetime(
    del_df["year"].astype(str) + del_df["quarter"].map(quarter_to_date)
).dt.strftime("%Y-%m-%d")

In [15]:
del_df.head()

,quarter,mortgage,heloc,auto,credit_card,student_loan,other,all,year,date
0,Q1,1.21,0.35,2.33,8.84,6.13,7.23,2.57,2003,2003-01-01
1,Q2,1.14,0.28,2.26,8.90,6.14,7.13,2.49,2003,2003-04-01
2,Q3,1.10,0.22,2.16,8.67,6.27,6.88,2.39,2003,2003-07-01
3,Q4,1.06,0.31,2.16,9.24,6.23,7.47,2.35,2003,2003-10-01
4,Q1,1.01,0.21,2.32,9.27,6.34,7.68,2.31,2004,2004-01-01


#### Export to DW

In [16]:
# https://app.datawrapper.de/chart/4xRqa/publish
#del_df_id = "4xRqa"
#dw.add_data(chart_id=f"{del_df_id}", data=del_df.query('year > "2014"'))

<Response [204]>

---

#### Total debt balance

#### By composition

In [17]:
debt_df = pd.read_excel(url, sheet_name="Page 3 Data", skiprows=3).rename(
    columns={"Unnamed: 0": "quarter"}
)
debt_df.columns = debt_df.columns.str.lower().str.replace(" ", "_")

In [18]:
debt_df["year"] = "20" + debt_df["quarter"].str.split(":", expand=True)[0]
debt_df["quarter"] = debt_df["quarter"].str.split(":", expand=True)[1]

In [19]:
debt_df["date"] = pd.to_datetime(
    debt_df["year"].astype(str) + debt_df["quarter"].map(quarter_to_date)
).dt.strftime("%Y-%m-%d")
debt_df["date"] = pd.to_datetime(debt_df["date"])

In [20]:
debt_df.head()

,quarter,mortgage,he_revolving,auto_loan,credit_card,student_loan,other,total,year,date
0,Q1,4.942,0.242,0.641,0.688,0.2407,0.4776,7.2313,2003,2003-01-01
1,Q2,5.080,0.260,0.622,0.693,0.2429,0.4860,7.3839,2003,2003-04-01
2,Q3,5.183,0.269,0.684,0.693,0.2488,0.4773,7.5551,2003,2003-07-01
3,Q4,5.660,0.302,0.704,0.698,0.2529,0.4486,8.0655,2003,2003-10-01
4,Q1,5.840,0.328,0.720,0.695,0.2598,0.4465,8.2893,2004,2004-01-01


#### Function to adjust for inflation

In [21]:
def adjust_for_inflation(row, column_name):
    # Adjusts to 2023 dollars
    year_of_data = row["date"].year
    return cpi.inflate(row[column_name], year_or_month=year_of_data, to=2023)

#### Apply inflation adjustment
> Use the non-housing measures. Other balances include retail cards and other consumer loans.

In [22]:
columns_to_adjust = [
    # "mortgage",
    # "he_revolving",
    "auto_loan",
    "credit_card",
    "student_loan",
    "other",
    # "total",
]

In [24]:
for column in columns_to_adjust:
    debt_df[column + "_infl_adj"] = debt_df[:84].apply(
        adjust_for_inflation, column_name=column, axis=1
    ).round(3)

In [30]:
debt_df[debt_df['date']>="2015-01-01"]

,quarter,mortgage,he_revolving,auto_loan,credit_card,student_loan,other,total,year,date,auto_loan_infl_adj,credit_card_infl_adj,student_loan_infl_adj,other_infl_adj
48,Q1,8.171,0.510,0.968,0.684,1.189,0.329,11.851,2015,2015-01-01,1.244,0.879,1.529,0.423
49,Q2,8.116,0.499,1.006,0.703,1.190,0.339,11.853,2015,2015-04-01,1.293,0.904,1.530,0.436
50,Q3,8.260,0.492,1.045,0.714,1.203,0.351,12.065,2015,2015-07-01,1.343,0.918,1.547,0.451
51,Q4,8.249,0.487,1.064,0.733,1.232,0.351,12.116,2015,2015-10-01,1.368,0.942,1.584,0.451
52,Q1,8.369,0.485,1.071,0.712,1.261,0.354,12.252,2016,2016-01-01,1.360,0.904,1.601,0.449
53,Q2,8.362,0.478,1.103,0.729,1.259,0.356,12.287,2016,2016-04-01,1.400,0.926,1.598,0.452
54,Q3,8.350,0.472,1.135,0.747,1.279,0.367,12.350,2016,2016-07-01,1.441,0.948,1.624,0.466
55,Q4,8.480,0.473,1.157,0.779,1.310,0.377,12.576,2016,2016-10-01,1.469,0.989,1.663,0.479
56,Q1,8.627,0.456,1.167,0.764,1.344,0.367,12.725,2017,2017-01-01,1.451,0.950,1.671,0.456
57,Q2,8.691,0.452,1.190,0.784,1.344,0.378,12.839,2017,2017-04-01,1.479,0.975,1.671,0.470


In [26]:
# https://app.datawrapper.de/chart/U2lZt/publish
#debt_df_id = "U2lZt"
#dw.add_data(chart_id=f"{debt_df_id}", data=debt_df.query('year > "2014"'))

<Response [204]>